**--- Part One ---**

In [119]:
## Select input data case
## Note: this assumes a plain textfile named `input_{case}` is located in the same folder as this notebook
case = "example" # <- input_example
case = "jfm"
case = "gatton"

# verbose output
verbose = False

import numpy as np
from itertools import cycle

# Read input
with open(f'input_{case}','r') as f:
    input_lines = f.readlines()

import numpy as np

dir_map = {'>':[0,1], '<':[0,-1], '^':[-1,0], 'v':[1,0]}

class Blizzard():
    def __init__(self,pos,val,grid):
        self.pos = np.array(pos)
        self.sym = val
        self.delta=dir_map[val]
        self.xmin = 1
        self.xmax = grid.shape[0]-2
        self.ymin = 1
        self.ymax = grid.shape[1]-2
        
    def move(self, grid):
        self.pos += self.delta
        i,j = self.pos
        if i<self.xmin:
            self.pos[0]=self.xmax
        elif i>self.xmax:
            self.pos[0] = self.xmin
        elif j<self.ymin:
            self.pos[1] = self.ymax
        elif j>self.ymax:
            self.pos[1] = self.ymin
        return self.pos

class Elf():
    def __init__(self, pos):
        self.pos = np.array(pos)
    
    def move_options(self, grid):
        # print('searching')
        options = []
        # s = self.pos+[1,0]
        # print(f'south: {s} ; val={grid[tuple(s)]}')
        if self.pos[0]<grid.shape[0]-1 and grid[tuple((s := self.pos+[1,0]))] == 0:
            options.append(s)
        if grid[tuple((e := self.pos+[0,1]))] == 0:
            options.append(e)
        if grid[tuple(self.pos)] == 0:
            options.append(self.pos)
        if grid[tuple((n := self.pos+[-1,0]))] == 0:
            options.append(n)
        if grid[tuple((w := self.pos+[0,-1]))] == 0:
            options.append(w)
        # print(options)
        return options

class Grid():
    def __init__(self, target='end',minute = 0):
        self.grid = np.zeros((len(input_lines), len(input_lines[0][:-1])))
        self.blizzards = []
        for i,row in enumerate(input_lines):
            for j,val in enumerate(row.strip()):
                if val == '#':
                    self.grid[i,j] = 1
                elif val in dir_map.keys():
                    self.blizzards.append(Blizzard((i,j), val, self.grid))
                    self.grid[i,j] = 1
        self.start = np.array((0,1))
        self.end = np.array((self.grid.shape[0]-1,self.grid.shape[1]-2))
        self.set_target(target)
        self.minute = minute

    def set_target(self,target):
        if target == 'end': 
            self.elves = [Elf(self.start)]
            self.target = self.end
        elif target == 'start': 
            self.target = self.start
            self.elves = [Elf(self.end)]

    def __getitem__(self,idx):
        return self.grid[idx]

    def move_all_blizzards(self):
        for b in self.blizzards:
            self.grid[tuple(b.pos)] -= 1
            b.move(self.grid)
            self.grid[tuple(b.pos)] += 1
    
    def move_elves(self):
        # print('moving elves')
        idx_del = []
        new_elves = []
        for idx,elf in enumerate(self.elves):
            options = elf.move_options(self.grid)
            if len(options) == 0:
                idx_del.append(idx)
            else:
                if len(options) > 1:
                    for opt in options[1:]:
                        new_elves.append(Elf(opt))
                elf.pos = options[0]
        # remove dead-ends
        self.elves = [elf for idx,elf in enumerate(self.elves) if idx not in idx_del]
        # add new paths
        self.elves.extend(new_elves)
        # remove duplicates
        collisions = set()
        uniq = []
        for elf in self.elves:
            i,j = elf.pos
            if (i,j) in collisions:
                continue
            collisions.add((i,j))
            uniq.append(elf)
        self.elves = uniq

    def execute_minute(self):
        self.move_all_blizzards()
        self.move_elves()
        self.minute += 1
    
    def execute_continuous(self):
        while not self.finished:
            self.execute_minute()

    @property
    def finished(self):
        for elf in self.elves:
            if np.all(elf.pos == self.target):
                return True
        else:
            return False

    def show(self):
        l = [list(row) for row in self.grid]
        #l = list(self.grid)
        for b in self.blizzards:
            i,j = b.pos
            if l[i][j] == 1:
                l[i][j] = b.sym
            else:
                l[i][j] = str(int(l[i][j]))
        for elf in self.elves:
            l[elf.pos[0]][elf.pos[1]] = 'E'
        for i,row in enumerate(l):
            for j,val in enumerate(row):
                if val == 1:
                    row[j] = '#'
                elif val == 0:
                    row[j] = '.'
            print(''.join([str(i) for i in row]))
       

In [120]:
grid = Grid()
grid.show()
print('')
grid.execute_continuous()
print(grid.minute)

#E########################################################################################################################
#>v^<>^^>.v<v^<>><>^v<>^.<v><>^<>^><><>v><^v>>><.<<<<^v.^<><^v>>^v^>v>.<^<^>^^^>^v>v><.v<v^>v<<.^^.<>vv<<.<>^^>vv^<<v>^v>#
#<<<v^>.><v^>.>^vv<<^^<><<<^^<v^<^<>^^.v..^<v^><^v.^<>vv<<.<<v>v^^v^^<v.<<^^>vv<>vvv^>^>.<<.vv<>^^><v>>v^^^<<^.^<>^.v><v<#
#>^v.>v<^^>^>v<<<<><>>vv<<>>v^>.<^v.>^>^^vv^<<v<^^vv><<v^<^v<^>>^.^v^^v^<<<^v>^<>^^^v.^>>>^^>>^<<.<<<<.><<><^.<.v.^v><vv<#
#<>>vvv^^v><<<vv.><.<>>><^<v.>v>v^v^>^^<>v><^>><v>^v<>>>v>^<<vvv^<v<^^<<v^>.^..<v<<<v>><vvvv<>.<^^.v^^<<<>><><>.vv><^>vv<#
#<v<v<vv>>><v<><<^>.><><v^^v>>vv><><v<v^v^<^^.>>>^>.^<<v^>>><^.<^<.><<vv^>^<vv>>^<>^v<^v^v^<>v<<^<.<><vv<.<v>^>v<^<v<^.v<#
#.^<v^>^v.^^><.vvv^^^>v<^v>.><v^>.>^^<^<^vv<<.<v<^<v^<<><<^vv^<<v<.>^.<<^v<>.<^<vv.>>v>>^v^>vvv<v<>v><>^^^v^>>.^vv<<.^v^>#
#.>^v^<>v<>>><<<^<^<v^<>.>^>v>v^^v<>^^v<>^<v^^^v.v^vvv^v<<v>v<>^^vv^v>><<.^>>v^^>^v>^v<<>.<^v<<^<<<.<<v^v<^v.v><>><<v<v<<#
#<><<<<><.^>v<^>

**--- Part Two ---**

In [121]:
minutes = []
grid = Grid()
grid.show()
print('')
grid.execute_continuous()
print(f'minutes to reach end: {grid.minute}')
grid.set_target('start')
grid.execute_continuous()
print(f'total minutes to return to start: {grid.minute}')
grid.set_target('end')
grid.execute_continuous()
print(f'total minutes to make two trips: {grid.minute}')


#E########################################################################################################################
#>v^<>^^>.v<v^<>><>^v<>^.<v><>^<>^><><>v><^v>>><.<<<<^v.^<><^v>>^v^>v>.<^<^>^^^>^v>v><.v<v^>v<<.^^.<>vv<<.<>^^>vv^<<v>^v>#
#<<<v^>.><v^>.>^vv<<^^<><<<^^<v^<^<>^^.v..^<v^><^v.^<>vv<<.<<v>v^^v^^<v.<<^^>vv<>vvv^>^>.<<.vv<>^^><v>>v^^^<<^.^<>^.v><v<#
#>^v.>v<^^>^>v<<<<><>>vv<<>>v^>.<^v.>^>^^vv^<<v<^^vv><<v^<^v<^>>^.^v^^v^<<<^v>^<>^^^v.^>>>^^>>^<<.<<<<.><<><^.<.v.^v><vv<#
#<>>vvv^^v><<<vv.><.<>>><^<v.>v>v^v^>^^<>v><^>><v>^v<>>>v>^<<vvv^<v<^^<<v^>.^..<v<<<v>><vvvv<>.<^^.v^^<<<>><><>.vv><^>vv<#
#<v<v<vv>>><v<><<^>.><><v^^v>>vv><><v<v^v^<^^.>>>^>.^<<v^>>><^.<^<.><<vv^>^<vv>>^<>^v<^v^v^<>v<<^<.<><vv<.<v>^>v<^<v<^.v<#
#.^<v^>^v.^^><.vvv^^^>v<^v>.><v^>.>^^<^<^vv<<.<v<^<v^<<><<^vv^<<v<.>^.<<^v<>.<^<vv.>>v>>^v^>vvv<v<>v><>^^^v^>>.^vv<<.^v^>#
#.>^v^<>v<>>><<<^<^<v^<>.>^>v>v^^v<>^^v<>^<v^^^v.v^vvv^v<<v>v<>^^vv^v>><<.^>>v^^>^v>^v<<>.<^v<<^<<<.<<v^v<^v.v><>><<v<v<<#
#<><<<<><.^>v<^>

In [107]:
grid.minute

18